In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import re

# url 과 QueryString 자동 추출 함수 

In [ ]:
def url_to_params(url):
  split_url = url.split('?')
  b_url = split_url[0]
  
  querystring = split_url[1].split('&')
  querystring2 = []
  params = {}
  for i in range(len(querystring)):
    querystring2.append(querystring[i].split('='))

  for i in range(len(querystring2)):
    params[querystring2[i][0]] = querystring2[i][1]

  return b_url ,params

# Musinsa Standard > 1위 [ 트레이닝/조거팬츠 ] 후기, 별점 전체 추출

In [ ]:
def pred_comment(goodsNo):
  b_url, b_params = url_to_params('https://goods.musinsa.com/api/goods/v1/review/goods/list?sort=new&selectedSimilarNo=0&page=1&goodsNo='+str(goodsNo))

  headers = {
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.57 Whale/3.14.133.23 Safari/537.36',
      'referer': 'https://store.musinsa.com/'
  }
  resp = requests.get(url=b_url, headers=headers, params=b_params)
  soup = BeautifulSoup(resp.text, 'lxml')
  tags = soup.select('#reviewListFragment > div.nslist_bottom > div.box_page_msg')[0].text.strip().replace(' 페이지 중 1 페이지', '')
  total_page = int(tags)

  comment = []
  star = []

  a_params = b_params
  for page in range(1, total_page + 1):
    if page==2:
      break
    a_params['page'] = str(page)

    resp = requests.get(url=b_url, headers=headers, params=b_params)
    soup = BeautifulSoup(resp.text, 'lxml')
    a_tags = soup.select('#reviewListFragment > div > div.review-contents > div.review-contents__text')
    b_tags = soup.select('#reviewListFragment > div > div.review-list__rating-wrap > span > span > span')

    for a, b in zip(a_tags, b_tags):
      comment.append(a.text)
      star.append(b['style'][7:-1])


  return comment, star

# Musinsa Standard > 1위 [ 트레이닝/조거팬츠 ] 상품번호 전체 추출

In [ ]:
# url = 'https://www.musinsa.com/category/003004?d_cat_cd=003004&brand=musinsastandard&rate=&page_kind=search&list_kind=small&sort=emt_high&sub_sort=&page=1&display_cnt=90&sale_goods=&group_sale=&kids=&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure='
url = 'https://www.musinsa.com/categories/item/003004?d_cat_cd=003004&brand=musinsastandard&rate=&page_kind=search&list_kind=small&sort=emt_high&sub_sort=&page=1&display_cnt=90&sale_goods=&group_sale=&kids=&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure='

url1, params  =  url_to_params(url)

resp = requests.get(url=url1,params = params)


soup = BeautifulSoup(resp.content,'lxml')
goods_num_tags = soup.select('div.list_img a')

goods_num = []

for num in goods_num_tags:
    goods_num.append(re.findall('[0-9]+',num['href'])[0])

In [ ]:
num

<a class="img-block" href="//www.musinsa.com/app/goods/2126167" name="goods_link" title="우먼즈 기모 스웨트 팬츠 [에그렛]">
<img alt="무신사 스탠다드(MUSINSA STANDARD) 우먼즈 기모 스웨트 팬츠 [에그렛]" class="lazyload lazy" data-original="//image.msscdn.net/images/goods_img/20210913/2126167/2126167_1_125.jpg" src="//image.msscdn.net/images/no_image_125.png"/>
</a>

In [ ]:
new_dic = {}

In [ ]:
for num in tqdm(goods_num):
    new_dic[num] = pred_comment(num)

In [ ]:
new_dic['1212777']

(['검정 회색 둘다 샀는데 회색이 세탁시 수축이 더되네요',
  '무릎도 잘 안나오고 전투용으로 입고 다니기 딱 좋습니다!',
  '제가 원라하는 기장감과 통이지만 두께가 조금 있어서 아쉬워요',
  '178,75 처음 m구매했다가 통이 너무 딱 맞는 느낌이라L로 교환했네요.L은 허리가 많이 큰 편이긴한데끈으로 묶으면 큰 문제는 없어요',
  '가격도 싸고 재질도 부드럽고 편해서 집 앞 나갈때 저주 입습니다.',
  '기본 무난하게 입고다니기 좋은 옷입니다 가격도 착해요!',
  '혼또니 스고이데스네 배송도 빠르고 재질도 좋아요',
  '사이즈랑 재질 색감 모두 마음에 들어서 잘 입을거 같아요',
  '너무 좋아요 기대한데로에요완전 좋아요마음에 들어요',
  '중학생 아들용 구매입니다 좀 크긴합니다만 입을 만 해요'],
 ['100', '100', '100', '100', '100', '100', '100', '80', '100', '80'])

# Musinsa Standard > 1위 [ 트레이닝/조거팬츠 ] 상품번호, 후기, 별점 합산

In [ ]:
data_dic = {
    'goods_num' : [],
    'comment' : [],
    'star' : []
}

In [ ]:
for num in goods_num:
  for comment, star in zip(new_dic[num][0], new_dic[num][1]):
    data_dic['goods_num'].append(num)
    data_dic['comment'].append(comment)
    data_dic['star'].append(star)

In [ ]:
df = pd.DataFrame(data_dic)

In [ ]:
df

,goods_num,comment,star
0,1212777,검정 회색 둘다 샀는데 회색이 세탁시 수축이 더되네요,100
1,1212777,무릎도 잘 안나오고 전투용으로 입고 다니기 딱 좋습니다!,100
2,1212777,제가 원라하는 기장감과 통이지만 두께가 조금 있어서 아쉬워요,100
3,1212777,"178,75 처음 m구매했다가 통이 너무 딱 맞는 느낌이라L로 교환했네요.L은 허리...",100
4,1212777,가격도 싸고 재질도 부드럽고 편해서 집 앞 나갈때 저주 입습니다.,100
...,...,...,...
895,2126167,저 완전 하비인데 잘맞구 색도 예쁘고 맘에들어여,100
896,2126167,바지가 너무 편해요 색깔도 나름 귀엽고 만족합니다,100
897,2126167,엄마 사드렸는데 너무 좋아하세요감사합니다. 잘 입을께요!,80
898,2126167,사이즈도 딱 좋고 두툼해서 겨울에 따듯하게 잘 입었습니다,100
